#### Consolida banco de dados ####
**Consolida banco de dados em ../data/DiscursosSenadores.sqlite. Feito em 21 jan 2025**

In [1]:
import sqlite3
import os

# Caminho para o banco de dados
DB_PATH = "../../data/DiscursosSenadores.sqlite"

def adicionar_coluna_texto_integral(db_path):
    """Adiciona a coluna TextoIntegral no banco de dados, se não existir."""
    conexao = sqlite3.connect(db_path)
    cursor = conexao.cursor()

    try:
        cursor.execute("ALTER TABLE Discursos ADD COLUMN TextoIntegral TEXT")
        conexao.commit()
    except sqlite3.OperationalError:
        print("A coluna TextoIntegral já existe no banco de dados.")

    conexao.close()

def atualizar_texto_integral(db_path):
    """Atualiza a coluna TextoIntegral no banco de dados com o conteúdo dos arquivos de texto."""
    conexao = sqlite3.connect(db_path)
    cursor = conexao.cursor()

    cursor.execute("SELECT CodigoPronunciamento, CaminhoArquivo FROM discursos WHERE DownloadConcluido = 1")
    discursos = cursor.fetchall()

    for codigo, caminho_arquivo in discursos:
        if not caminho_arquivo or not os.path.exists(caminho_arquivo):
            continue

        try:
            with open(caminho_arquivo, "r", encoding="utf-8") as file:
                texto_integral = file.read()

            cursor.execute(
                """
                UPDATE Discursos
                SET TextoIntegral = ?
                WHERE CodigoPronunciamento = ?
                """,
                (texto_integral, codigo)
            )
        except Exception as e:
            print(f"Erro ao atualizar TextoIntegral para discurso {codigo}: {e}")

    conexao.commit()
    conexao.close()



A coluna TextoIntegral já existe no banco de dados.


KeyboardInterrupt: 

*Atualizando o banco de dados, coluna TextoIntegral*

In [2]:
def atualizar_banco_de_dados(db_path, temp_json_path):
    """Atualiza o banco de dados com os textos e caminhos do JSON temporário."""
    with open(temp_json_path, "r", encoding="utf-8") as json_file:
        temp_results = json.load(json_file)

    conexao = sqlite3.connect(db_path)
    cursor = conexao.cursor()

    for codigo, dados in temp_results.items():
        try:
            cursor.execute(
                """
                UPDATE discursos
                SET TextoIntegral = ?, CaminhoArquivo = ?
                WHERE CodigoPronunciamento = ?
                """,
                (dados["TextoIntegral"], dados["CaminhoArquivo"], codigo)
            )
            print(f"Banco de dados atualizado para Código: {codigo}")
        except Exception as e:
            print(f"Erro ao atualizar banco de dados para Código {codigo}: {e}")

    conexao.commit()
    conexao.close()

DB_PATH = "../../data/DiscursosSenadores.sqlite"
TEMP_JSON_PATH = "./temp/transcricoes_temp.json"

atualizar_banco_de_dados(DB_PATH, TEMP_JSON_PATH)


Banco de dados atualizado para Código: 469534
Banco de dados atualizado para Código: 469722
Banco de dados atualizado para Código: 472065
Banco de dados atualizado para Código: 469095
Banco de dados atualizado para Código: 466166
Banco de dados atualizado para Código: 480057
Banco de dados atualizado para Código: 482047
Banco de dados atualizado para Código: 469793
Banco de dados atualizado para Código: 470201
Banco de dados atualizado para Código: 469547
Banco de dados atualizado para Código: 470192
Banco de dados atualizado para Código: 463665
Banco de dados atualizado para Código: 469107
Banco de dados atualizado para Código: 469113
Banco de dados atualizado para Código: 469729
Banco de dados atualizado para Código: 469677
Banco de dados atualizado para Código: 468424
Banco de dados atualizado para Código: 482000


In [3]:
print("Tabela de discursos concluída")

Tabela de discursos concluída


*Incluir cargos nas tabelas de Senadores*

In [6]:
import re
import json
from xml.etree.ElementTree import fromstring

# Caminho para o arquivo de entrada e saída
arquivo_entrada = "../../data/senadores/cargos.xml"
arquivo_saida = "../../data/senadores/cargos_senadores.json"

# Ler o arquivo completo como texto
with open(arquivo_entrada, 'r', encoding='utf-8') as file:
    conteudo = file.read()

# Separar os blocos usando o padrão de comentário como delimitador
blocos = re.split(r'<!-- Senador: (\d+) -->', conteudo)

# A lista para armazenar os dados processados
dados_senadores = []

# Processar pares de código e XML
for i in range(1, len(blocos), 2):  # Começa do índice 1, pula de 2 em 2
    codigo_parlamentar = blocos[i]
    xml_conteudo = blocos[i + 1].strip()

    try:
        # Parsear o XML do bloco
        root = fromstring(xml_conteudo)

        # Extrair informações do senador
        parlamentar = root.find(".//Parlamentar")
        nome_senador = parlamentar.find("Nome").text if parlamentar.find("Nome") is not None else None

        # Processar os cargos
        cargos = []
        for cargo in parlamentar.findall(".//Cargo"):  # Localiza todos os elementos <Cargo>
            descricao_cargo = cargo.find("DescricaoCargo").text if cargo.find("DescricaoCargo") is not None else None
            data_inicio = cargo.find("DataInicio").text if cargo.find("DataInicio") is not None else None
            data_fim = cargo.find("DataFim").text if cargo.find("DataFim") is not None else None
            orgao = cargo.find("IdentificacaoComissao/NomeComissao").text if cargo.find("IdentificacaoComissao/NomeComissao") is not None else None
            id_cargo = cargo.find("IdCargo").text if cargo.find("CodigoCargo") is not None else None

            cargos.append({
                "DescricaoCargo": descricao_cargo,
                "DataInicio": data_inicio,
                "DataFim": data_fim,
                "Orgao": orgao,
                "IdCargo": id_cargo,
            })

        # Adicionar os dados do senador à lista
        dados_senadores.append({
            "CodigoParlamentar": codigo_parlamentar,
            "Nome": nome_senador,
            "Cargos": cargos
        })

    except Exception as e:
        print(f"Erro ao processar Senador {codigo_parlamentar}: {e}")

# Salvar os dados em um arquivo JSON
with open(arquivo_saida, 'w', encoding='utf-8') as json_file:
    json.dump(dados_senadores, json_file, ensure_ascii=False, indent=4)

print(f"Dados salvos em {arquivo_saida}")


Erro ao processar Senador 5672: 'NoneType' object has no attribute 'text'
Erro ao processar Senador 5982: 'NoneType' object has no attribute 'text'
Erro ao processar Senador 6358: 'NoneType' object has no attribute 'text'
Erro ao processar Senador 5967: 'NoneType' object has no attribute 'text'
Erro ao processar Senador 6009: 'NoneType' object has no attribute 'text'
Erro ao processar Senador 6350: 'NoneType' object has no attribute 'text'
Erro ao processar Senador 4639: 'NoneType' object has no attribute 'text'
Erro ao processar Senador 6295: 'NoneType' object has no attribute 'text'
Erro ao processar Senador 5936: 'NoneType' object has no attribute 'text'
Erro ao processar Senador 5990: 'NoneType' object has no attribute 'text'
Erro ao processar Senador 470: 'NoneType' object has no attribute 'text'
Erro ao processar Senador 5973: 'NoneType' object has no attribute 'text'
Erro ao processar Senador 739: 'NoneType' object has no attribute 'text'
Erro ao processar Senador 475: 'NoneType

*Acrescentar ao banco de dados*

In [7]:
# Inserir os dados no banco de dados SQLite
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Adicionar a coluna Orgao se não existir
cursor.execute("PRAGMA table_info(SenadoresCargos)")
colunas_existentes = [col[1] for col in cursor.fetchall()]

if "Orgao" not in colunas_existentes:
    cursor.execute("ALTER TABLE SenadoresCargos ADD COLUMN Orgao TEXT")
    print("Coluna 'Orgao' adicionada à tabela SenadoresCargos.")

# Inserir os dados do JSON na tabela
for senador in dados_senadores:
    for cargo in senador["Cargos"]:
        cursor.execute('''
        INSERT INTO SenadoresCargos (CodigoParlamentar, NomeCargo, DataInicio, DataFim, Orgao, IdCargo)
        VALUES (?, ?, ?, ?, ?)
        ''', (
            senador["CodigoParlamentar"],
            cargo["DescricaoCargo"],
            cargo["DataInicio"],
            cargo["DataFim"],
            cargo["Orgao"],
            cargo["IdCargo"]
        ))

# Confirmar e fechar a conexão
conn.commit()
conn.close()

print("Dados inseridos no banco de dados com sucesso.")


Dados inseridos no banco de dados com sucesso.


*Agora, fazer a mesma coisa com histórico acadêmico*

In [19]:
import re
import json
import sqlite3
from xml.etree.ElementTree import fromstring

# Caminho para os arquivos
arquivo_entrada = "../../data/senadores/historico_academico.xml"
arquivo_saida = "../../data/senadores/historico_academico.json"
banco_dados = "../../data/DiscursosSenadores.sqlite"

# Ler o arquivo completo como texto
with open(arquivo_entrada, 'r', encoding='utf-8') as file:
    conteudo = file.read()

# Separar os blocos usando o padrão de comentário como delimitador
blocos = re.split(r'<!-- Senador: (\d+) -->', conteudo)

# A lista para armazenar os dados processados
dados_historico = []

# Processar pares de código e XML
for i in range(1, len(blocos), 2):  # Começa do índice 1, pula de 2 em 2
    codigo_parlamentar = blocos[i]
    xml_conteudo = blocos[i + 1].strip()

    try:
        # Parsear o XML do bloco
        root = fromstring(xml_conteudo)

        # Extrair histórico acadêmico
        historico = []
        for curso in root.findall(".//Curso"):
            nome_curso = curso.find("NomeCurso").text if curso.find("NomeCurso") is not None else None
            nivel_curso = curso.find("NivelCurso").text if curso.find("NivelCurso") is not None else None
            instituicao = curso.find("InstituicaoEnsino").text if curso.find("InstituicaoEnsino") is not None else None
            ano_conclusao = curso.find("AnoConclusao").text if curso.find("AnoConclusao") is not None else None

            historico.append({
                "NomeCurso": nome_curso,
                "NivelCurso": nivel_curso,
                "InstituicaoEnsino": instituicao,
                "AnoConclusao": ano_conclusao
            })

        # Adicionar os dados do senador à lista
        dados_historico.append({
            "CodigoParlamentar": codigo_parlamentar,
            "Historico": historico
        })

    except Exception as e:
        print(f"Erro ao processar Senador {codigo_parlamentar}: {e}")

# Salvar os dados em um arquivo JSON
with open(arquivo_saida, 'w', encoding='utf-8') as json_file:
    json.dump(dados_historico, json_file, ensure_ascii=False, indent=4)

print(f"Dados salvos em {arquivo_saida}")

Dados salvos em ../data/senadores/historico_academico.json


*Atualizar o banco de dados*

In [20]:
# Inserir os dados no banco de dados SQLite
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Inserir os dados do JSON na tabela
for senador in dados_historico:
    for curso in senador["Historico"]:
        cursor.execute('''
        INSERT INTO SenadoresHistoricoAcademico (CodigoParlamentar, NomeCurso, NivelCurso, InstituicaoEnsino, AnoConclusao)
        VALUES (?, ?, ?, ?, ?)
        ''', (
            senador["CodigoParlamentar"],
            curso["NomeCurso"],
            curso["NivelCurso"],
            curso["InstituicaoEnsino"],
            curso["AnoConclusao"]
        ))

# Confirmar e fechar a conexão
conn.commit()
conn.close()

print("Dados inseridos no banco de dados com sucesso.")

Dados inseridos no banco de dados com sucesso.


*Incluir lideranças*

In [8]:
import re
import json
import sqlite3
from xml.etree.ElementTree import fromstring, ElementTree

# Caminho para os arquivos
arquivo_entrada = "../../data/senadores/liderancas.xml"
arquivo_saida = "../../data/senadores/liderancas.json"
banco_dados = "../../data/DiscursosSenadores.sqlite"

# Ler o arquivo completo como texto
with open(arquivo_entrada, 'r', encoding='utf-8') as file:
    conteudo = file.read()

# Separar os blocos usando o padrão de comentário como delimitador
blocos = re.split(r'<!-- Senador: (\d+) -->', conteudo)

# A lista para armazenar os dados processados
dados_liderancas = []

# Processar pares de código e XML
for i in range(1, len(blocos), 2):  # Começa do índice 1, pula de 2 em 2
    codigo_parlamentar = blocos[i]
    xml_conteudo = blocos[i + 1].strip()

    try:
        # Debug: Mostrar os primeiros caracteres do XML para verificar o conteúdo
        print(f"Processando Senador {codigo_parlamentar}. Prévia do XML: {xml_conteudo[:100]}")

        # Parsear o XML do bloco
        root = fromstring(xml_conteudo)

        # Extrair lideranças
        liderancas = []
        parlamentar = root.find(".//Parlamentar")
        if parlamentar is not None:
            liderancas_element = parlamentar.find("Liderancas")
            if liderancas_element is not None:
                for lideranca in liderancas_element.findall("Lideranca"):
                    unidade_lideranca = lideranca.find("UnidadeLideranca")
                    descricao_tipo_lideranca = lideranca.find("DescricaoTipoLideranca")
                    data_inicio = lideranca.find("DataDesignacao")
                    data_fim = lideranca.find("DataFim")

                    # Adicionar os valores à lista
                    liderancas.append({
                        "UnidadeLideranca": unidade_lideranca.text if unidade_lideranca is not None else None,
                        "DescricaoTipoLideranca": descricao_tipo_lideranca.text if descricao_tipo_lideranca is not None else None,
                        "DataInicio": data_inicio.text if data_inicio is not None else None,
                        "DataFim": data_fim.text if data_fim is not None else None,
                    })

        # Adicionar os dados do senador à lista
        dados_liderancas.append({
            "CodigoParlamentar": codigo_parlamentar,
            "Liderancas": liderancas
        })

    except Exception as e:
        print(f"Erro ao processar Senador {codigo_parlamentar}: {e}")

# Salvar os dados em um arquivo JSON
with open(arquivo_saida, 'w', encoding='utf-8') as json_file:
    json.dump(dados_liderancas, json_file, ensure_ascii=False, indent=4)

print(f"Dados salvos em {arquivo_saida}")


Processando Senador 5672. Prévia do XML: <?xml version='1.0' encoding='UTF-8'?>
<LiderancaParlamentar xmlns:xsi='http://www.w3.org/2001/XMLSc
Processando Senador 5982. Prévia do XML: <?xml version='1.0' encoding='UTF-8'?>
<LiderancaParlamentar xmlns:xsi='http://www.w3.org/2001/XMLSc
Processando Senador 6358. Prévia do XML: <?xml version='1.0' encoding='UTF-8'?>
<LiderancaParlamentar xmlns:xsi='http://www.w3.org/2001/XMLSc
Processando Senador 6366. Prévia do XML: <?xml version='1.0' encoding='UTF-8'?>
<LiderancaParlamentar xmlns:xsi='http://www.w3.org/2001/XMLSc
Processando Senador 5967. Prévia do XML: <?xml version='1.0' encoding='UTF-8'?>
<LiderancaParlamentar xmlns:xsi='http://www.w3.org/2001/XMLSc
Processando Senador 751. Prévia do XML: <?xml version='1.0' encoding='UTF-8'?>
<LiderancaParlamentar xmlns:xsi='http://www.w3.org/2001/XMLSc
Processando Senador 6009. Prévia do XML: <?xml version='1.0' encoding='UTF-8'?>
<LiderancaParlamentar xmlns:xsi='http://www.w3.org/2001/XMLSc
Process

*Insere lideranças no banco de dados*

In [9]:
import sqlite3
import json

# Caminho para o arquivo JSON e o banco de dados
arquivo_json = "../../data/senadores/liderancas.json"
banco_dados = "../../data/DiscursosSenadores.sqlite"

# Ler o arquivo JSON
with open(arquivo_json, 'r', encoding='utf-8') as json_file:
    dados_liderancas = json.load(json_file)

# Conectar ao banco de dados
conn = sqlite3.connect(banco_dados)
cursor = conn.cursor()



# Inserir os dados do JSON na tabela
for senador in dados_liderancas:
    codigo_parlamentar = senador["CodigoParlamentar"]
    for lideranca in senador["Liderancas"]:
        cursor.execute('''
        INSERT INTO SenadoresLiderancas (CodigoParlamentar, UnidadeLideranca, DescricaoTipoLideranca, DataInicio, DataFim)
        VALUES (?, ?, ?, ?, ?)
        ''', (
            codigo_parlamentar,
            lideranca["UnidadeLideranca"],
            lideranca["DescricaoTipoLideranca"],
            lideranca["DataInicio"],
            lideranca["DataFim"]
        ))

# Confirmar e fechar a conexão
conn.commit()
conn.close()

print("Dados de lideranças inseridos no banco de dados com sucesso.")


OperationalError: database is locked

*Incluir comissões*

In [10]:
import sqlite3
import re
import json
from xml.etree.ElementTree import fromstring

# Caminhos para os arquivos
arquivo_xml = "../../data/senadores/comissoes.xml"
arquivo_json = "../../data/senadores/comissoes.json"
banco_dados = "../../data/DiscursosSenadores.sqlite"

# Ler o arquivo XML
with open(arquivo_xml, 'r', encoding='utf-8') as file:
    conteudo = file.read()

# Separar os blocos usando o padrão de comentário como delimitador
blocos = re.split(r'<!-- Senador: (\d+) -->', conteudo)

# A lista para armazenar os dados processados
dados_comissoes = []

# Processar pares de código e XML
for i in range(1, len(blocos), 2):  # Começa do índice 1, pula de 2 em 2
    codigo_parlamentar = blocos[i]
    xml_conteudo = blocos[i + 1].strip()

    try:
        # Parsear o XML do bloco
        root = fromstring(xml_conteudo)

        # Extrair as comissões
        comissoes = []
        comissoes_element = root.find(".//MembroComissoes")
        if comissoes_element is not None:
            for comissao in comissoes_element.findall("Comissao"):
                identificacao = comissao.find("IdentificacaoComissao")
                sigla_comissao = identificacao.find("SiglaComissao") if identificacao is not None else None
                nome_comissao = identificacao.find("NomeComissao") if identificacao is not None else None
                descricao_participacao = comissao.find("DescricaoParticipacao")
                data_inicio = comissao.find("DataInicio")
                data_fim = comissao.find("DataFim")

                # Adicionar os valores à lista
                comissoes.append({
                    "SiglaComissao": sigla_comissao.text if sigla_comissao is not None else None,
                    "NomeComissao": nome_comissao.text if nome_comissao is not None else None,
                    "DescricaoParticipacao": descricao_participacao.text if descricao_participacao is not None else None,
                    "DataInicio": data_inicio.text if data_inicio is not None else None,
                    "DataFim": data_fim.text if data_fim is not None else None,
                })

        # Adicionar os dados do senador à lista
        dados_comissoes.append({
            "CodigoParlamentar": codigo_parlamentar,
            "Comissoes": comissoes
        })

    except Exception as e:
        print(f"Erro ao processar Senador {codigo_parlamentar}: {e}")

# Salvar os dados em um arquivo JSON
with open(arquivo_json, 'w', encoding='utf-8') as json_file:
    json.dump(dados_comissoes, json_file, ensure_ascii=False, indent=4)

print(f"Dados salvos em {arquivo_json}")

Dados salvos em ../../data/senadores/comissoes.json


*Inserir comissões no banco de dados*

In [11]:
# Conectar ao banco de dados
conn = sqlite3.connect(banco_dados)
cursor = conn.cursor()


# Inserir os dados do JSON na tabela
for senador in dados_comissoes:
    codigo_parlamentar = senador["CodigoParlamentar"]
    for comissao in senador["Comissoes"]:
        cursor.execute('''
        INSERT INTO SenadoresComissoes (CodigoParlamentar, SiglaComissao, NomeComissao, DescricaoParticipacao, DataInicio, DataFim)
        VALUES (?, ?, ?, ?, ?, ?)
        ''', (
            codigo_parlamentar,
            comissao["SiglaComissao"],
            comissao["NomeComissao"],
            comissao["DescricaoParticipacao"],
            comissao["DataInicio"],
            comissao["DataFim"]
        ))

# Confirmar e fechar a conexão
conn.commit()
conn.close()

print("Dados de comissões inseridos no banco de dados com sucesso.")


Dados de comissões inseridos no banco de dados com sucesso.


*Agora, filiações.*

In [12]:
import sqlite3
import re
import json
from xml.etree.ElementTree import fromstring

# Caminhos para os arquivos
arquivo_xml = "../../data/senadores/filiacoes.xml"
arquivo_json = "../../data/senadores/filiacoes.json"
banco_dados = "../../data/DiscursosSenadores.sqlite"

# Ler o arquivo XML
with open(arquivo_xml, 'r', encoding='utf-8') as file:
    conteudo = file.read()

# Separar os blocos usando o padrão de comentário como delimitador
blocos = re.split(r'<!-- Senador: (\d+) -->', conteudo)

# A lista para armazenar os dados processados
dados_filiacoes = []

# Processar pares de código e XML
for i in range(1, len(blocos), 2):  # Começa do índice 1, pula de 2 em 2
    codigo_parlamentar = blocos[i]
    xml_conteudo = blocos[i + 1].strip()

    try:
        # Parsear o XML do bloco
        root = fromstring(xml_conteudo)

        # Extrair as filiações
        filiacoes = []
        filiacoes_element = root.find(".//Filiacoes")
        if filiacoes_element is not None:
            for filiacao in filiacoes_element.findall("Filiacao"):
                partido = filiacao.find("Partido")
                sigla_partido = partido.find("SiglaPartido") if partido is not None else None
                nome_partido = partido.find("NomePartido") if partido is not None else None
                data_filiacao = filiacao.find("DataFiliacao")

                # Adicionar os valores à lista
                filiacoes.append({
                    "SiglaPartido": sigla_partido.text if sigla_partido is not None else None,
                    "NomePartido": nome_partido.text if nome_partido is not None else None,
                    "DataFiliacao": data_filiacao.text if data_filiacao is not None else None,
                })

        # Adicionar os dados do senador à lista
        dados_filiacoes.append({
            "CodigoParlamentar": codigo_parlamentar,
            "Filiacoes": filiacoes
        })

    except Exception as e:
        print(f"Erro ao processar Senador {codigo_parlamentar}: {e}")

# Salvar os dados em um arquivo JSON
with open(arquivo_json, 'w', encoding='utf-8') as json_file:
    json.dump(dados_filiacoes, json_file, ensure_ascii=False, indent=4)

print(f"Dados salvos em {arquivo_json}")

Dados salvos em ../../data/senadores/filiacoes.json


*Inserir filiações no banco de dados*

In [13]:
# Conectar ao banco de dados
conn = sqlite3.connect(banco_dados)
cursor = conn.cursor()

# Inserir os dados do JSON na tabela
for senador in dados_filiacoes:
    codigo_parlamentar = senador["CodigoParlamentar"]
    for filiacao in senador["Filiacoes"]:
        cursor.execute('''
        INSERT INTO SenadoresFiliacoes (CodigoParlamentar, SiglaPartido, NomePartido, DataFiliacao)
        VALUES (?, ?, ?, ?)
        ''', (
            codigo_parlamentar,
            filiacao["SiglaPartido"],
            filiacao["NomePartido"],
            filiacao["DataFiliacao"]
        ))

# Confirmar e fechar a conexão
conn.commit()
conn.close()

print("Dados de filiações inseridos no banco de dados com sucesso.")


Dados de filiações inseridos no banco de dados com sucesso.


## Banco de dados DiscursosSenadores.sqlite concluído! ##